In [1]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob

### 데이터 수집

In [2]:
ticker_nm = '005930'
# ticker_nm = '247540'
start_date  = '20190101'
today_date1 = '20231017'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

### 이동평균선 계산

In [3]:
df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

### 볼린저밴드 계산

In [4]:
std = df_raw['close'].rolling(20).std(ddof=0)

df_raw['upper'] = df_raw['MA20'] + 2 * std
df_raw['lower'] = df_raw['MA20'] - 2 * std

### MACD 계산

In [5]:
# MACD 
macd = MACD(close=df_raw['close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)


df_raw['MACD_DIFF'] = macd.macd_diff()
df_raw['MACD'] = macd.macd()
df_raw['MACD_Signal'] = macd.macd_signal()

### RSI 계산

 - [rsi 계산](https://wikidocs.net/163550)

In [6]:
df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

# welles moving average
df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

In [8]:

down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]

down_reg_df = pd.DataFrame({
    'index':down_reg,
    'name':'매도'})

top_reg_df = pd.DataFrame({
    'index':top_reg,
    'name':'매수'})
    
cross_df = pd.concat([down_reg_df, top_reg_df])
cross_df = cross_df.reset_index(drop = True)

In [22]:
df_raw['RSI_test'] = '-'
# df_raw['RSI_test'][down_reg] = '매도'
# df_raw['RSI_test'][top_reg] = '매수'

df_raw.loc[down_reg, 'RSI_test'] = '매도타이밍'
df_raw.loc[top_reg, 'RSI_test'] = '매수타이밍'

In [23]:
df_raw

,date,open,high,low,close,volume,price_change_percentage,ticker,MA5,MA20,...,MACD_DIFF,MACD,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,RSI_test
0,2019-01-02,39400,39400,38550,38750,7847664,0.129199,005930,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,-
1,2019-01-03,38300,38550,37450,37600,12471493,-2.967742,005930,NaN,NaN,...,NaN,NaN,NaN,-1150.0,0.0,1150.0,NaN,NaN,NaN,-
2,2019-01-04,37450,37600,36850,37450,14108958,-0.398936,005930,NaN,NaN,...,NaN,NaN,NaN,-150.0,0.0,150.0,NaN,NaN,NaN,-
3,2019-01-07,38000,38900,37800,38750,12748997,3.471295,005930,NaN,NaN,...,NaN,NaN,NaN,1300.0,1300.0,0.0,NaN,NaN,NaN,-
4,2019-01-08,38000,39200,37950,38100,12756554,-1.677419,005930,38130.0,NaN,...,NaN,NaN,NaN,-650.0,0.0,650.0,NaN,NaN,NaN,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,2023-10-11,68600,69400,67900,68200,25209349,2.710843,005930,66960.0,69255.0,...,-325.379314,-598.874845,-273.495532,1800.0,1800.0,0.0,329.454457,365.345458,47.417170,-
1178,2023-10-12,68600,69700,68200,68900,19311380,1.026393,005930,67240.0,69180.0,...,-170.292627,-486.361316,-316.068688,700.0,700.0,0.0,355.921995,339.249354,51.199175,-
1179,2023-10-13,68000,68500,67700,68000,9724086,-1.306241,005930,67500.0,69065.0,...,-118.714635,-464.461983,-345.747347,-900.0,0.0,900.0,330.498996,379.302972,46.562141,-
1180,2023-10-16,67900,68500,66800,67300,12599299,-1.029412,005930,67760.0,68890.0,...,-121.683584,-497.851827,-376.168243,-700.0,0.0,700.0,306.891925,402.209902,43.278964,-


In [16]:
df_raw['RSI_test'][down_reg]

16      0
26      0
120     0
176     0
181     0
207     0
237     0
251     0
256     0
349     0
389     0
462     0
467     0
470     0
474     0
481     0
490     0
737     0
1004    0
1087    0
1091    0
Name: RSI_test, dtype: object

,date,open,high,low,close,volume,price_change_percentage,ticker,MA5,MA20,...,lower,MACD_DIFF,MACD,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI
0,2019-01-02,39400,39400,38550,38750,7847664,0.129199,005930,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,2019-01-03,38300,38550,37450,37600,12471493,-2.967742,005930,NaN,NaN,...,NaN,NaN,NaN,NaN,-1150.0,0.0,1150.0,NaN,NaN,NaN
2,2019-01-04,37450,37600,36850,37450,14108958,-0.398936,005930,NaN,NaN,...,NaN,NaN,NaN,NaN,-150.0,0.0,150.0,NaN,NaN,NaN
3,2019-01-07,38000,38900,37800,38750,12748997,3.471295,005930,NaN,NaN,...,NaN,NaN,NaN,NaN,1300.0,1300.0,0.0,NaN,NaN,NaN
4,2019-01-08,38000,39200,37950,38100,12756554,-1.677419,005930,38130.0,NaN,...,NaN,NaN,NaN,NaN,-650.0,0.0,650.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,2023-10-11,68600,69400,67900,68200,25209349,2.710843,005930,66960.0,69255.0,...,65942.916064,-325.379314,-598.874845,-273.495532,1800.0,1800.0,0.0,329.454457,365.345458,47.417170
1178,2023-10-12,68600,69700,68200,68900,19311380,1.026393,005930,67240.0,69180.0,...,65907.325253,-170.292627,-486.361316,-316.068688,700.0,700.0,0.0,355.921995,339.249354,51.199175
1179,2023-10-13,68000,68500,67700,68000,9724086,-1.306241,005930,67500.0,69065.0,...,65796.192878,-118.714635,-464.461983,-345.747347,-900.0,0.0,900.0,330.498996,379.302972,46.562141
1180,2023-10-16,67900,68500,66800,67300,12599299,-1.029412,005930,67760.0,68890.0,...,65636.755466,-121.683584,-497.851827,-376.168243,-700.0,0.0,700.0,306.891925,402.209902,43.278964


In [16]:
df_raw = df_raw[df_raw['date'] > '2023-01-01']
df_raw = df_raw.reset_index(drop = True)

### 함수

In [20]:
def macd_vis(df_raw):
    # fig = go.Figure()
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.4, 0.2, 0.2, 0.2])
#     fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.6, 0.4])    

    # 캔들스틱차트
    fig.add_trace(go.Candlestick(
        x=df_raw['date'],
        open=df_raw['open'],
        high=df_raw['high'],
        low=df_raw['low'],
        close=df_raw['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = ''), row = 1, col = 1)

    # MA 5 
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA5'],
                             opacity=0.7,
                             line=dict(color='blue', width=2),
                             name='MA 5') , row = 1, col = 1)

    # MA 20
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA20'],
                             opacity=0.7,
                             line=dict(color='orange', width=2),
                             name='MA 20'), row = 1, col = 1)

    # MA 60
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA60'],
                             opacity=0.7,
                             line=dict(color='purple', width=2),
                             name='MA 60'), row = 1, col = 1)

    # MA 120
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA120'],
                             opacity=0.7,
                             line=dict(color='green', width=2),
                             name='MA 120'), row = 1, col = 1)


    fig.add_trace(go.Scatter(
        x=pd.concat([df_raw['date'], df_raw['date'][::-1]]),
        y=pd.concat([df_raw['upper'], df_raw['lower'][::-1]]),
        fill='toself',
        fillcolor='rgba(255,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0.2)', width=2),
        name='Bollinger Band',
        showlegend=False
    ), row = 1, col = 1)

    # 상향, 하향 회귀
    for i in range(len(cross_df)):
        cross_index = cross_df['index'][i]
        cross_name = cross_df['name'][i]

        cross_date = df_raw['date'][cross_index]
        cross_value = df_raw['close'][cross_index]

        fig.add_annotation(x=cross_date, 
                           y=cross_value,
                           text=cross_name,
                           showarrow=True,
                           arrowhead=1,
                           row = 1, col = 1)


    # Row 2 
    # volume
    colors = ['blue' if row['open'] - row['close'] >= 0 
              else 'red' for index, row in df_raw.iterrows()]

    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['volume'],
                         marker_color=colors,
                         name = 'Volume',
                         showlegend=False
                        ), row=2, col=1)

    # MACD
    colors = ['red' if val >= 0 
              else 'blue' for val in df_raw['MACD_DIFF']]
    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['MACD_DIFF'],
                         marker_color=colors,
                         name =  'MACD DIFF'
                        ), row=3, col=1)

    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD'],
                             line=dict(color='green', width=2),
                             name = 'MACD'
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD_Signal'],
                             line=dict(color='orange', width=1),
                             name = 'MACD Signal'
                            ), row=3, col=1)

    # Row 5
    # RSI
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['RSI'],
                             line=dict(color='red', width=1),
                             name = 'RSI'
                            ), row=4, col=1)
    
    # 수평 사각 영역 추가하기
    
    fig.add_hrect(y0=70, y1=100, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매수구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="red",
                  annotation_font_family="Times New Roman", row=4, col=1)
    
    fig.add_hrect(y0=0, y1=30, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매도 구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="blue",
                  annotation_font_family="Times New Roman", row=4, col=1)    
    
#     fig.add_hrect(y0=40, y1=60, line_width=1, fillcolor="blue", opacity=0.1,
#                   annotation_text=" ", 
#                   annotation_position="top left",
#                   annotation_font_size=20,
#                   annotation_font_color="red",
#                   annotation_font_family="Times New Roman", row=4, col=1)
    
    
    # Rayout
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=100  #top margin
        ),
        height=400, width=900, 
    #     showlegend=False, 
        xaxis_rangeslider_visible=False
    )


    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="RSI", row=4, col=1)

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    return fig

In [21]:
macd_vis(df_raw)

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

